In [1]:
import os
import time
import zipfile
import numpy as np
import random

from sklearn.decomposition import PCA
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import os
import time
import math



# Set global random seeds
random.seed(42)
np.random.seed(42)

# Set environment variables for deterministic behavior
os.environ['PYTHONHASHSEED'] = '42'
os.environ['OMP_NUM_THREADS'] = '1'  # For OpenMP
os.environ['MKL_NUM_THREADS'] = '1'  # For MKL
os.environ['OPENBLAS_NUM_THREADS'] = '1'  # For OpenBLAS
os.environ['NUMEXPR_NUM_THREADS'] = '1'  # For NumExpr
#os.environ['VECLIB_MAXIMUM_THREADS'] = '1'  # For macOS Accelerate

from sklearn.feature_extraction.text import CountVectorizer


def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        dirs.sort()  # Sort directories to ensure consistent traversal
        files.sort()  # Sort files to ensure consistent order
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")

    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")

    return dataPointsList

###############################################################################
# Data Extraction and Vectorization

# Parameters setup
flakyZip = 'Dataset/flaky_files.zip'
nonFlakyZip = 'Dataset/reduced_nonflaky_files.zip'
imbalanceNonFlakyZip = 'Dataset/nonflaky_files.zip'

# Create directories
outDirbalance = "results/balance_flaky_nonflaky/"
outDirimbalance = "results/imbalance_nonflaky/"
os.makedirs(outDirbalance, exist_ok=True)
os.makedirs(outDirimbalance, exist_ok=True)

extractDirbalance = "extracted/balance_flaky_nonflaky/"
extractDirimbalance = "extracted/imbalance_nonflaky/"
os.makedirs(extractDirbalance, exist_ok=True)
os.makedirs(extractDirimbalance, exist_ok=True)

# Extract and read data once for balance combination
flakyDirbalance = os.path.join(extractDirbalance, 'flaky')
nonFlakyDirbalance = os.path.join(extractDirbalance, 'nonFlaky')
os.makedirs(flakyDirbalance, exist_ok=True)
os.makedirs(nonFlakyDirbalance, exist_ok=True)

extract_zip(flakyZip, flakyDirbalance)
extract_zip(nonFlakyZip, nonFlakyDirbalance)

dataPointsFlakybalance = getDataPoints(flakyDirbalance)
dataPointsNonFlakybalance = getDataPoints(nonFlakyDirbalance)
dataPointsbalance = dataPointsFlakybalance + dataPointsNonFlakybalance

# Print the number of datasets for balance combination
print(f"Number of flaky documents (balance combination): {len(dataPointsFlakybalance)}")
print(f"Number of non-flaky documents (balance combination): {len(dataPointsNonFlakybalance)}")
print(f"Total number of documents (balance combination): {len(dataPointsbalance)}")

dataLabelsListbalance = np.array([1]*len(dataPointsFlakybalance) + [0]*len(dataPointsNonFlakybalance))

# Extract and read data once for imbalance non-flaky combination
flakyDirimbalance = os.path.join(extractDirimbalance, 'flaky')
nonFlakyDirimbalance = os.path.join(extractDirimbalance, 'nonFlaky')
os.makedirs(flakyDirimbalance, exist_ok=True)
os.makedirs(nonFlakyDirimbalance, exist_ok=True)

extract_zip(flakyZip, flakyDirimbalance)
extract_zip(imbalanceNonFlakyZip, nonFlakyDirimbalance)

dataPointsFlakyimbalance = getDataPoints(flakyDirimbalance)
dataPointsNonFlakyimbalance = getDataPoints(nonFlakyDirimbalance)
dataPointsimbalance = dataPointsFlakyimbalance + dataPointsNonFlakyimbalance

# Print the number of datasets for imbalance combination
print(f"Number of flaky documents (imbalance combination): {len(dataPointsFlakyimbalance)}")
print(f"Number of non-flaky documents (imbalance combination): {len(dataPointsNonFlakyimbalance)}")
print(f"Total number of documents (imbalance combination): {len(dataPointsimbalance)}")

dataLabelsListimbalance = np.array([1]*len(dataPointsFlakyimbalance) + [0]*len(dataPointsNonFlakyimbalance))


Number of flaky documents (balance combination): 45
Number of non-flaky documents (balance combination): 45
Total number of documents (balance combination): 90
Number of flaky documents (imbalance combination): 45
Number of non-flaky documents (imbalance combination): 243
Total number of documents (imbalance combination): 288


## RF

In [2]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef, make_scorer)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

###############################################################################
# Custom MCC scorer function
def mcc_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

###############################################################################
# Random Forest

def runRandomForest(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()

    # Define scoring metrics
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer
    }

    # Define a pipeline with CountVectorizer and Random Forest
    pipeline = Pipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),  # Vectorizer
        ('rf', RandomForestClassifier(random_state=42))    # Random Forest classifier
    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
        'rf__n_estimators': [50, 100, 200],                 # Number of trees in the forest
        'rf__max_depth': [10, 20, 30],                      # Maximum depth of the tree
        'rf__min_samples_split': [5, 10],                   # Minimum number of samples required to split a node
        'rf__min_samples_leaf': [2, 5],                     # Minimum number of samples required at a leaf node
        'rf__criterion': ['gini', 'entropy'],               # Function to measure the quality of a split
        'rf__class_weight': ['balanced']                    # Class weights
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=skf, scoring=scoring, refit='f1', verbose=1, return_train_score=True
    )

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    print(f"Best Parameters: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results and print final metrics
    results = grid_search.cv_results_

    # Prepare per-fold metrics for CSV along with the parameter combinations
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'n_estimators': results['params'][idx].get('rf__n_estimators'),
            'max_depth': results['params'][idx].get('rf__max_depth'),
            'min_samples_split': results['params'][idx].get('rf__min_samples_split'),
            'min_samples_leaf': results['params'][idx].get('rf__min_samples_leaf'),
            'criterion': results['params'][idx].get('rf__criterion'),
            'class_weight': results['params'][idx].get('rf__class_weight'),
            'accuracy': results.get('mean_test_accuracy', [None])[idx],
            'std_accuracy': results.get('std_test_accuracy', [None])[idx],
            'precision': results.get('mean_test_precision', [None])[idx],
            'std_precision': results.get('std_test_precision', [None])[idx],
            'recall': results.get('mean_test_recall', [None])[idx],
            'std_recall': results.get('std_test_recall', [None])[idx],
            'f1': results.get('mean_test_f1', [None])[idx],
            'std_f1': results.get('std_test_f1', [None])[idx],
            'mcc': results.get('mean_test_mcc', [None])[idx],
            'std_mcc': results.get('std_test_mcc', [None])[idx],
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile_folds = os.path.join(outDir, f"rf-fold-results-folds.csv")
    df_folds.to_csv(outFile_folds, index=False)

    print(f"Per-fold metrics saved to: {outFile_folds}")

    # Extract final metrics based on the cross-validation results
    best_index = grid_search.best_index_
    final_f1 = results['mean_test_f1'][best_index]
    std_final_f1 = results['std_test_f1'][best_index]
    final_precision = results['mean_test_precision'][best_index]
    std_final_precision = results['std_test_precision'][best_index]
    final_recall = results['mean_test_recall'][best_index]
    std_final_recall = results['std_test_recall'][best_index]
    final_accuracy = results['mean_test_accuracy'][best_index]
    std_final_accuracy = results['std_test_accuracy'][best_index]
    final_mcc = results['mean_test_mcc'][best_index]
    std_final_mcc = results['std_test_mcc'][best_index]

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision} (Std Dev: {std_final_precision})")
    print(f"Final Recall: {final_recall} (Std Dev: {std_final_recall})")
    print(f"Final Accuracy: {final_accuracy} (Std Dev: {std_final_accuracy})")
    print(f"Final F1 Score: {final_f1} (Std Dev: {std_final_f1})")
    print(f"Final MCC: {final_mcc} (Std Dev: {std_final_mcc})")

    # Save the results to a CSV file
    outFile_final = os.path.join(outDir, f"rf-results.csv")
    with open(outFile_final, "w") as f:
        f.write("Accuracy,Std_Accuracy,Precision,Std_Precision,Recall,Std_Recall,F1,Std_F1,MCC,Std_MCC\n")
        f.write(f"{final_accuracy},{std_final_accuracy},{final_precision},{std_final_precision},"
                f"{final_recall},{std_final_recall},{final_f1},{std_final_f1},{final_mcc},{std_final_mcc}\n")

    print(f"Random Forest analysis completed for {n_splits}-folds. Results saved to: {outFile_final}")

    # Return the best parameters and metrics with their standard deviations
    return best_params, final_f1, std_final_f1, final_mcc, std_final_mcc

###############################################################################
# Main Execution for 5-Fold Cross-Validation

outDir = "results"
os.makedirs(outDir, exist_ok=True)

# Run Random Forest on balance combination
print("\nStarting Random Forest analysis for flaky vs smaller non-flaky files (balance combination)...")
best_params_5folds_balance, best_score_5folds_balance, std_f1_balance, balance_mcc_5folds, std_mcc_balance = runRandomForest(
    dataPointsbalance, dataLabelsListbalance, outDirbalance, 5
)

print("\nBest results for Random Forest 5-fold on balance combination:")
print(f"Best Parameters: {best_params_5folds_balance}")
print(f"Best F1 Score: {best_score_5folds_balance} (Std Dev: {std_f1_balance})")
print(f"Final MCC: {balance_mcc_5folds} (Std Dev: {std_mcc_balance})")

# Run Random Forest on imbalance combination
print("\nStarting Random Forest analysis for flaky vs imbalance non-flaky files (imbalance combination)...")
best_params_5folds_imbalance, best_f1_5folds, std_f1_imbalance, final_mcc_5folds, std_mcc_imbalance = runRandomForest(
    dataPointsimbalance, dataLabelsListimbalance, outDirimbalance, 5
)

# Display results
print("\nBest results for Random Forest 5-fold on imbalance combination:")
print(f"Best Parameters: {best_params_5folds_imbalance}")
print(f"Best F1 Score: {best_f1_5folds} (Std Dev: {std_f1_imbalance})")
print(f"Final MCC: {final_mcc_5folds} (Std Dev: {std_mcc_imbalance})")



Starting Random Forest analysis for flaky vs smaller non-flaky files (balance combination)...
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters: {'rf__class_weight': 'balanced', 'rf__criterion': 'entropy', 'rf__max_depth': 10, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 5, 'rf__n_estimators': 200}
Best F1 Score from cross-validation: 0.8908359133126936
Per-fold metrics saved to: results/balance_flaky_nonflaky/rf-fold-results-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.8786363636363637 (Std Dev: 0.07076325329243224)
Final Recall: 0.9111111111111111 (Std Dev: 0.08314794192830981)
Final Accuracy: 0.8888888888888887 (Std Dev: 0.0496903994999953)
Final F1 Score: 0.8908359133126936 (Std Dev: 0.050435935677112964)
Final MCC: 0.7856438407434342 (Std Dev: 0.1001822478018566)
Random Forest analysis completed for 5-folds. Results saved to: results/balance_flaky_nonflaky/rf-results.csv

Best results for Random Forest 5-fold on balance combinat

## Decision Tree

In [2]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef, make_scorer)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.feature_extraction.text import CountVectorizer

###############################################################################
# Custom MCC scorer function
def mcc_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

###############################################################################
# Decision Tree

def runDecisionTree(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()

    # Define scoring metrics
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer
    }

    # Define a pipeline and Decision Tree
    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),  # Vectorizer
        ('dt', DecisionTreeClassifier(random_state=42))    # Decision Tree classifier
    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
        'dt__max_depth': [10, 20, 30],                     # Maximum depth of the tree
        'dt__min_samples_split': [5, 10],                  # Minimum number of samples required to split a node
        'dt__min_samples_leaf': [2, 5],                    # Minimum number of samples required at a leaf node
        'dt__criterion': ['gini', 'entropy'],              # Function to measure the quality of a split
        'dt__max_features': [None, 'sqrt', 'log2'],        # Controls how many features to consider for splits
        'dt__class_weight': ['balanced']                   # Class weights
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=skf, scoring=scoring,
        refit='f1', verbose=1, return_train_score=True
    )

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    # Get the index of the best parameter combination
    best_index = grid_search.best_index_

    # Extract the cross-validation results
    results = grid_search.cv_results_

    # Extract standard deviations of the best scores
    std_f1_best = results['std_test_f1'][best_index]
    std_mcc_best = results['std_test_mcc'][best_index]

    print(f"Best Parameters: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv} (Std Dev: {std_f1_best})")

    # Prepare per-fold metrics for CSV along with the parameter combinations
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'max_depth': results['params'][idx].get('dt__max_depth'),
            'min_samples_split': results['params'][idx].get('dt__min_samples_split'),
            'min_samples_leaf': results['params'][idx].get('dt__min_samples_leaf'),
            'criterion': results['params'][idx].get('dt__criterion'),
            'max_features': results['params'][idx].get('dt__max_features'),
            'class_weight': results['params'][idx].get('dt__class_weight'),
            'accuracy': results.get('mean_test_accuracy', [None])[idx],
            'std_accuracy': results.get('std_test_accuracy', [None])[idx],
            'precision': results.get('mean_test_precision', [None])[idx],
            'std_precision': results.get('std_test_precision', [None])[idx],
            'recall': results.get('mean_test_recall', [None])[idx],
            'std_recall': results.get('std_test_recall', [None])[idx],
            'f1': results.get('mean_test_f1', [None])[idx],
            'std_f1': results.get('std_test_f1', [None])[idx],
            'mcc': results.get('mean_test_mcc', [None])[idx],
            'std_mcc': results.get('std_test_mcc', [None])[idx],
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile_folds = os.path.join(outDir, f"dt-results.csv")
    df_folds.to_csv(outFile_folds, index=False)

    print(f"Per-fold metrics saved to: {outFile_folds}")

    # Extract final metrics based on the cross-validation results
    final_f1 = results['mean_test_f1'][best_index]
    std_final_f1 = results['std_test_f1'][best_index]
    final_precision = results['mean_test_precision'][best_index]
    std_final_precision = results['std_test_precision'][best_index]
    final_recall = results['mean_test_recall'][best_index]
    std_final_recall = results['std_test_recall'][best_index]
    final_accuracy = results['mean_test_accuracy'][best_index]
    std_final_accuracy = results['std_test_accuracy'][best_index]
    final_mcc = results['mean_test_mcc'][best_index]
    std_final_mcc = results['std_test_mcc'][best_index]

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision} (Std Dev: {std_final_precision})")
    print(f"Final Recall: {final_recall} (Std Dev: {std_final_recall})")
    print(f"Final Accuracy: {final_accuracy} (Std Dev: {std_final_accuracy})")
    print(f"Final F1 Score: {final_f1} (Std Dev: {std_final_f1})")
    print(f"Final MCC: {final_mcc} (Std Dev: {std_final_mcc})")

    # Save the results to a CSV file
    outFile_final = os.path.join(outDir, f"dt-final.csv")
    with open(outFile_final, "w") as f:
        f.write("Accuracy,Std_Accuracy,Precision,Std_Precision,Recall,Std_Recall,F1,Std_F1,MCC,Std_MCC\n")
        f.write(f"{final_accuracy},{std_final_accuracy},{final_precision},{std_final_precision},{final_recall},{std_final_recall},{final_f1},{std_final_f1},{final_mcc},{std_final_mcc}\n")

    print(f"Decision Tree analysis completed for {n_splits}-folds. Results saved to: {outFile_final}")

    # Return the best parameters and metrics with their standard deviations
    return best_params, final_f1, std_final_f1, final_mcc, std_final_mcc

###############################################################################
# Main Execution for 5-Fold Cross-Validation

outDir = "results"
os.makedirs(outDir, exist_ok=True)

# Run Decision Tree on balance combination
print("\nStarting Decision Tree analysis for flaky vs smaller non-flaky files (balance combination)...")
best_params_5folds_balance, best_f1_score_balance, std_f1_balance, best_mcc_balance, std_mcc_balance = runDecisionTree(
    dataPointsbalance, dataLabelsListbalance, outDirbalance, 5
)

print("\nBest results for Decision Tree 5-fold on balance combination:")
print(f"Best Parameters: {best_params_5folds_balance}")
print(f"Best F1 Score: {best_f1_score_balance} (Std Dev: {std_f1_balance})")
print(f"Best MCC Score: {best_mcc_balance} (Std Dev: {std_mcc_balance})")

# Run Decision Tree on imbalance non-flaky combination
print("\nStarting Decision Tree analysis for flaky vs imbalance non-flaky files (imbalance combination)...")
best_params_5folds_imbalance, best_f1_score_imbalance, std_f1_imbalance, best_mcc_imbalance, std_mcc_imbalance = runDecisionTree(
    dataPointsimbalance, dataLabelsListimbalance, outDirimbalance, 5
)

print("\nBest results for Decision Tree 5-fold on imbalance combination:")
print(f"Best Parameters: {best_params_5folds_imbalance}")
print(f"Best F1 Score: {best_f1_score_imbalance} (Std Dev: {std_f1_imbalance})")
print(f"Best MCC Score: {best_mcc_imbalance} (Std Dev: {std_mcc_imbalance})")



Starting Decision Tree analysis for flaky vs smaller non-flaky files (balance combination)...
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters: {'dt__class_weight': 'balanced', 'dt__criterion': 'entropy', 'dt__max_depth': 10, 'dt__max_features': None, 'dt__min_samples_leaf': 5, 'dt__min_samples_split': 5}
Best F1 Score from cross-validation: 0.8768716577540108 (Std Dev: 0.08813638922817157)
Per-fold metrics saved to: results/balance_flaky_nonflaky/dt-results.csv

Final Cross-Validation Metrics:
Final Precision: 0.9098901098901099 (Std Dev: 0.12205224946738305)
Final Recall: 0.8666666666666666 (Std Dev: 0.12957670877434002)
Final Accuracy: 0.8777777777777779 (Std Dev: 0.08888888888888888)
Final F1 Score: 0.8768716577540108 (Std Dev: 0.08813638922817157)
Final MCC: 0.7764255562923227 (Std Dev: 0.16215494031026728)
Decision Tree analysis completed for 5-folds. Results saved to: results/balance_flaky_nonflaky/dt-final.csv

Best results for Decision Tree 5-fold